In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import torch.optim as optim
import pandas as pd
import numpy as np

In [19]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"

class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size) - torch.eye(input_size, input_size)
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size))
        self.b = Parameter(torch.Tensor(output_size))
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size)
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

In [3]:
class FCRBFGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FCRBFGRU, self).__init__()

        self.RBFtemp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.RBftemp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.RBFtemp_decay_r = TemporalDecay(input_size, input_size, diag = True)

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)

        self.concat_temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.RBFoutput_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)

        self.RBFbeta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.RBFgrucell = nn.GRUCell(self.input_size * 2, self.hidden_size)

        self.concoat_beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.concat_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.hidden_dropout = nn.Dropout(p=0.5)

        self.diag = torch.ones(self.input_size,self.input_size).cuda() - torch.eye(self.input_size,self.input_size).cuda()
        self.W = torch.nn.parameter.Parameter(torch.Tensor(self.input_size, self.input_size))
        self.linear2 = nn.Linear(self.input_size, self.input_size, bias = True)
        self.linear3 = nn.Linear(self.input_size, self.input_size, bias = True)
        self.V = torch.nn.parameter.Parameter(torch.Tensor(self.input_size, self.input_size))
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        RBF_hid = torch.zeros((values.size(0), self.hidden_size)).cuda()
        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        rbf_loss = 0.0
        gru_loss = 0.0
        concat_loss = 0.0

        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            # RBF temporal
            RBF_gamma_x = self.RBftemp_decay_x(d)
            
            RBF_gamma_h = self.RBFtemp_decay_h(d)

            r_hat = self.RBFtemp_decay_r(r)
            rbf_loss += torch.sum(torch.abs(v - r_hat) * m) / (torch.sum(m) + 1e-5)
            
            RBF_x_hat = self.RBFoutput_layer(RBF_hid)
            rbf_loss += torch.sum(torch.abs(v - RBF_x_hat) * m) / (torch.sum(m) + 1e-5)

            RBF_beta_weight = torch.cat([RBF_gamma_x, m], dim = 1)
            RBF_beta = torch.sigmoid(self.RBFbeta_layer(RBF_beta_weight))

            RBF_c_hat = RBF_beta * r_hat + (1 - RBF_beta) * RBF_x_hat
            rbf_loss += torch.sum(torch.abs(v - RBF_c_hat) * m) / (torch.sum(m) + 1e-5)

            RBF_c_c = m * v + (1 - m) * RBF_c_hat

            RBF_gru_input = torch.cat([RBF_c_c, m], dim = 1)
            
            # RBF GRU cell
            RBF_hid = RBF_hid * RBF_gamma_h
            self.hidden_dropout(RBF_hid)
            RBF_hid = self.RBFgrucell(RBF_gru_input, RBF_hid)

            # MGRU
            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)

            x_hat = self.output_layer(hid)
            gru_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)

            x_c = m * v + (1 - m) * x_hat
            
            z_hat = self.z_layer(x_c)
            gru_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)
            
            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            gru_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            
            # GRU cell
            hid = hid * gamma_h
            hid = self.grucell(gru_input, hid)

            # concat_RBF_GRU

            concat_gamma_x = self.concat_temp_decay_x(d)

            RG = torch.cat([RBF_c_hat, c_hat], dim = 1)
            concat = self.concat_layer(RG)
            concat_loss += torch.sum(torch.abs(v - concat) * m) / (torch.sum(m) + 1e-5)

            concat_c = m * v + (1 - m) * concat
            concat_z_hat = self.linear2(concat_c) + torch.matmul(v, self.diag * self.W)

            concat_loss += torch.sum(torch.abs(v - concat_z_hat) * m) / (torch.sum(m) + 1e-5)

            concoat_beta_weight = torch.cat([concat_gamma_x, m], dim = 1)
            concat_beta = torch.sigmoid(self.concoat_beta_layer(concoat_beta_weight))

            concat_c_hat = concat_beta * concat_z_hat + (1 - concat_beta) * concat
            concat_loss += torch.sum(torch.abs(v - concat_c_hat) * m) / (torch.sum(m) + 1e-5)

            concoat_c_c = m * v + (1 - m) * concat_c_hat

            h_t = torch.matmul(concoat_c_c, self.V)
  
            imputation_X = self.linear3(h_t)

            concat_loss += torch.sum(torch.abs(v - imputation_X) * m) / (torch.sum(m) + 1e-5)

            imputation_X_c = m * v + (1 - m) * imputation_X
            imputations.append(imputation_X_c.unsqueeze_(1))

            c_hat_list.append(imputation_X.unsqueeze(1))
        

        x_loss = rbf_loss + gru_loss + concat_loss

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)

        return imputations, x_loss, c_hat_list

In [4]:
input_size = 36
hidden_size = 64

In [35]:

RBFoutput_layer = nn.Linear(hidden_size, input_size, bias=True)
output_layer = nn.Linear(hidden_size, input_size, bias=True)

z_layer = FeatureRegression(input_size)
beta_layer = nn.Linear(input_size * 2, input_size)
grucell = nn.GRUCell(input_size * 2, hidden_size)

RBFbeta_layer = nn.Linear(input_size * 2, input_size)
RBFgrucell = nn.GRUCell(input_size * 2, hidden_size)

concoat_beta_layer = nn.Linear(input_size * 2, input_size)
concat_layer = nn.Linear(input_size * 2, input_size)
hidden_dropout = nn.Dropout(p=0.5)

diag = torch.ones(input_size,input_size) - torch.eye(input_size,input_size)
W = torch.nn.parameter.Parameter(torch.Tensor(input_size, input_size))
linear2 = nn.Linear(input_size, input_size, bias = True)
linear3 = nn.Linear(input_size, input_size, bias = True)
V = torch.nn.parameter.Parameter(torch.Tensor(input_size, input_size))

In [36]:
RBFtemp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
RBftemp_decay_x = TemporalDecay(input_size, input_size, diag = True)
RBFtemp_decay_r = TemporalDecay(input_size, input_size, diag = True)

temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
temp_decay_x = TemporalDecay(input_size, input_size, diag = True)

concat_temp_decay_x = TemporalDecay(input_size, input_size, diag = True)


In [37]:
from dataloader import missing_data_rbf, eval_model, eval_bi_model

In [38]:
df = pd.read_csv("./dataset/"+"pm25_missing.csv").drop(["datetime"], axis = 1)
dataset = missing_data_rbf(df, "air_1000_0.05_time.csv", 64, 2)

In [39]:
c = 0
for data in dataset:
    if c == 0:
        input = data
    c +=1 

In [40]:
values = input[:,0,::]
delta = input[:,1,::]
masks = input[:,2,::]
rbfs = input[:,3,::]

RBF_hid = torch.zeros((values.size(0), hidden_size))
hid = torch.zeros((values.size(0), hidden_size))

In [41]:
v = values[:,0,:]
d = delta[:,0,:]
m = masks[:,0,:]
r = rbfs[:,0,:]
rbf_loss = 0.0
# RBF temporal
RBF_gamma_x = RBftemp_decay_x(d)

RBF_gamma_h = RBFtemp_decay_h(d)

r_hat = RBFtemp_decay_r(r)
rbf_loss += torch.sum(torch.abs(v - r_hat) * m) / (torch.sum(m) + 1e-5)

RBF_x_hat = RBFoutput_layer(RBF_hid)
rbf_loss += torch.sum(torch.abs(v - RBF_x_hat) * m) / (torch.sum(m) + 1e-5)

RBF_beta_weight = torch.cat([RBF_gamma_x, m], dim = 1)
RBF_beta = torch.sigmoid(RBFbeta_layer(RBF_beta_weight))

RBF_c_hat = RBF_beta * r_hat + (1 - RBF_beta) * RBF_x_hat
rbf_loss += torch.sum(torch.abs(v - RBF_c_hat) * m) / (torch.sum(m) + 1e-5)

RBF_c_c = m * v + (1 - m) * RBF_c_hat

RBF_gru_input = torch.cat([RBF_c_c, m], dim = 1)

In [42]:
RBF_c_hat[3]

tensor([0.3426, 0.4037, 0.0880, 0.3042, 0.5413, 0.5409, 0.7487, 0.2959, 0.4872,
        0.3634, 0.3295, 0.3762, 0.7436, 0.5878, 0.6051, 0.2241, 0.2420, 0.3545,
        0.6326, 0.4130, 0.6317, 0.5260, 0.3214, 0.4381, 0.2434, 0.5335, 0.6288,
        0.4104, 0.7181, 0.2662, 0.5824, 0.4076, 0.4526, 0.3790, 0.6003, 0.4115],
       grad_fn=<SelectBackward0>)

In [43]:
rbf_loss

tensor(3.0457, grad_fn=<AddBackward0>)

In [44]:
# MGRU
gru_loss = 0.0
gamma_x = temp_decay_x(d)
gamma_h = temp_decay_h(d)

x_hat = output_layer(hid)
gru_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)

x_c = m * v + (1 - m) * x_hat

z_hat = z_layer(x_c)
gru_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

beta_weight = torch.cat([gamma_x, m], dim = 1)
beta = torch.sigmoid(beta_layer(beta_weight))

c_hat = beta * z_hat + (1 - beta) * x_hat
gru_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

c_c = m * v + (1 - m) * c_hat

gru_input = torch.cat([c_c, m], dim = 1)

In [45]:
c_hat

tensor([[-0.1691, -0.3117, -0.0144,  ...,  0.1093, -0.2849,  0.3670],
        [ 0.0107, -0.0603,  0.0208,  ...,  0.0133,  0.0015,  0.0251],
        [-0.0157, -0.0072,  0.0683,  ...,  0.1284,  0.1019,  0.0204],
        ...,
        [ 0.0301,  0.1064,  0.1902,  ...,  0.1319,  0.1425,  0.0457],
        [-0.1753, -0.1091, -0.3963,  ..., -0.3496, -0.5075,  0.2824],
        [-0.0336, -0.0377,  0.0830,  ...,  0.0933,  0.0820,  0.0337]],
       grad_fn=<AddBackward0>)

In [50]:
from models.model import MGRU,BiMGRU, train_MGRU, train_BiMGRU, RBF_MGRU, BiRBFMGRU, FCRBFGRU

In [47]:
# concat_RBF_GRU
concat_loss = 0.0
concat_gamma_x = concat_temp_decay_x(d)

RG = torch.cat([RBF_c_hat, c_hat], dim = 1)
concat = concat_layer(RG)
concat_loss += torch.sum(torch.abs(v - concat) * m) / (torch.sum(m) + 1e-5)

concat_c = m * v + (1 - m) * concat
concat_z_hat = linear2(concat_c) + torch.matmul(v, diag * W)

concat_loss += torch.sum(torch.abs(v - concat_z_hat) * m) / (torch.sum(m) + 1e-5)

concoat_beta_weight = torch.cat([concat_gamma_x, m], dim = 1)
concat_beta = torch.sigmoid(concoat_beta_layer(concoat_beta_weight))

concat_c_hat = concat_beta * concat_z_hat + (1 - concat_beta) * concat
concat_loss += torch.sum(torch.abs(v - concat_c_hat) * m) / (torch.sum(m) + 1e-5)

concoat_c_c = m * v + (1 - m) * concat_c_hat

h_t = torch.matmul(concoat_c_c, V)

imputation_X = linear3(h_t)

concat_loss += torch.sum(torch.abs(v - imputation_X) * m) / (torch.sum(m) + 1e-5)

imputation_X_c = m * v + (1 - m) * imputation_X

In [48]:
concat_loss

tensor(17.4496, grad_fn=<AddBackward0>)

In [49]:
model = FCRBFGRU(input_size, hidden_size)

In [53]:
model = FCRBFGRU(input_size, hidden_size)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model,"air_1000_0.05_time.csv","pm25_ground.csv","pm25_missing.csv")

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor(0.8376, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6015, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3947, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1978, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.0481, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.8214, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.6245, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.4391, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.2512, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.9974, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8.7668, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.5441, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.3478, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.0907, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.8521, device='cuda:0', grad_fn=<AddBackward0>)
tensor(12.6244, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.4261, device='cuda:0', grad_fn=<AddBackward0>)
tensor(14.1719, device='cuda:0', grad_fn=<A

loss: 1684.5130615234375:   0%|          | 1/1000 [00:02<49:07,  2.95s/it]

tensor(0.7270, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3160, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.9460, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6363, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.3904, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9910, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.6395, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.3554, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.0725, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.6572, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.2792, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.9624, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8.6572, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.2389, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.8489, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.5135, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.2392, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.8468, device='cuda:0', grad_fn=<AddB

loss: 1472.85693359375:   0%|          | 2/1000 [00:05<47:53,  2.88s/it]  

tensor(0.7258, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2407, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8085, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4807, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2385, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7855, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.3777, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.0797, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.8189, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.3672, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.9540, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.6438, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8.3494, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8.8915, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.4653, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.1280, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.8800, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.4410, device='cuda:0', grad_fn=<AddB

loss: 1472.85693359375:   0%|          | 2/1000 [00:07<58:47,  3.53s/it]

tensor(36.6369, device='cuda:0', grad_fn=<AddBackward0>)
tensor(37.2330, device='cuda:0', grad_fn=<AddBackward0>)
tensor(37.9387, device='cuda:0', grad_fn=<AddBackward0>)
tensor(38.7456, device='cuda:0', grad_fn=<AddBackward0>)
tensor(39.3375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(39.9724, device='cuda:0', grad_fn=<AddBackward0>)
tensor(40.7090, device='cuda:0', grad_fn=<AddBackward0>)
tensor(41.4802, device='cuda:0', grad_fn=<AddBackward0>)
tensor(42.0688, device='cuda:0', grad_fn=<AddBackward0>)
tensor(42.6836, device='cuda:0', grad_fn=<AddBackward0>)
tensor(43.3900, device='cuda:0', grad_fn=<AddBackward0>)
tensor(44.1757, device='cuda:0', grad_fn=<AddBackward0>)
tensor(44.7555, device='cuda:0', grad_fn=<AddBackward0>)
tensor(45.3704, device='cuda:0', grad_fn=<AddBackward0>)
tensor(46.0962, device='cuda:0', grad_fn=<AddBackward0>)
tensor(46.8544, device='cuda:0', grad_fn=<AddBackward0>)
tensor(47.4222, device='cuda:0', grad_fn=<AddBackward0>)
tensor(48.0197, device='cuda:0'

KeyboardInterrupt: 

In [52]:
len(dataset)

3